In [26]:
import pandas as pd
import numpy as np
import datetime as dt


Cargamos las bases de datos

In [39]:
dir_data = "./airbnb_final_csv/"
calendar = pd.read_csv(f"{dir_data}calendar_fin.csv")
listings = pd.read_csv(f"{dir_data}listings_fin.csv")
neighbourhoods = pd.read_csv(f"{dir_data}neighbourhoods_fin.csv")
reviews = pd.read_csv(f"{dir_data}reviews_fin.csv")
reviews_gz = pd.read_csv(f"{dir_data}reviews_gz_fin.csv")
listings_gz = pd.read_csv(f"{dir_data}listings_gz_fin.csv")

KeyboardInterrupt: 

Procesamos los datos para poder cargarlos a la base de datos. Empezamos con dos datos de listings_gz

In [63]:
# Proceso las columnas que deberian que son booleanas y tienen "f" y "t" como valor:

listings_gz["host_is_superhost"] = listings_gz["host_is_superhost"].replace({"f":0, "t":1, })
listings_gz["host_has_profile_pic"] = listings_gz["host_has_profile_pic"].replace({"f":0, "t":1, })
listings_gz["host_identity_verified"] = listings_gz["host_identity_verified"].replace({"f":0, "t":1, })

# Tratamos los valores nulos:

print(listings_gz["host_is_superhost"])
print(listings_gz["host_has_profile_pic"])
print(listings_gz["host_has_profile_pic"])

0        0.0
1        0.0
2        0.0
3        NaN
4        0.0
        ... 
93596    0.0
93597    0.0
93598    0.0
93599    0.0
93600    0.0
Name: host_is_superhost, Length: 93601, dtype: float64
0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
93596    1.0
93597    1.0
93598    1.0
93599    1.0
93600    1.0
Name: host_has_profile_pic, Length: 93601, dtype: float64
0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
93596    1.0
93597    1.0
93598    1.0
93599    1.0
93600    1.0
Name: host_has_profile_pic, Length: 93601, dtype: float64


Ahora vamos con las variables numericas

In [64]:
# Empezamos quitandole el simbolo del porcentaje y diviendiendo entre 100 a las variables host_response_rate y a host_
# Para ello utilizamos una funcion:

def conversion_porcentajes(name):
    name = str(name).replace("%","")
    name = float(name)/100
    return name

listings_gz["host_response_rate"] = listings_gz["host_response_rate"].apply(conversion_porcentajes)
listings_gz["host_acceptance_rate"] = listings_gz["host_acceptance_rate"].apply(conversion_porcentajes)

In [65]:
# Los nombres de las columnas correctas, EN ORDEN
proc_host_column_names_new = ['id', 'url', 'name', 'host_since', 'location', 'host_about', 'response_time', 'response_rate','acceptance_rate', 'is_superhost', 
                            'thumbnail_url', 'picture_url', 'listings_count', 'total_listings_count', 'verifications', 'has_profile_pic','identity_verified', 
                            'calculated_host_listings_count','calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
                            'calculated_host_listings_count_shared_rooms']

# Creamos el dataframe, EN EL ORDEN ANTERIOR
proc_hosts = listings_gz[["host_id", "host_url", "host_name", "host_since", "host_location", "host_about", "host_response_time", "host_response_rate", "host_acceptance_rate", 
                        "host_is_superhost", "host_thumbnail_url", "host_picture_url", "host_listings_count", "host_total_listings_count", "host_verifications",
                        "host_has_profile_pic", "host_identity_verified", "calculated_host_listings_count", "calculated_host_listings_count_entire_homes",
                        "calculated_host_listings_count_private_rooms", "calculated_host_listings_count_shared_rooms"]]

#Con esto renombro las columnas
proc_hosts = proc_hosts.set_axis(proc_host_column_names_new, axis=1)

# Esto es una prueba para ver si sale bien
proc_hosts.to_csv('proc_host.csv', index=False)

In [173]:
# Create dataframes from SQL tables
# Calendar table
proc_calendar = pd.DataFrame(columns=[
    'listing_id', 'date', 'available', 'price', 'adjusted_price', 'minimum_nights', 'maximum_nights'
])

# Property type table
proc_property_type = pd.DataFrame(columns=['id', 'type'])

# Listings table
proc_listings = pd.DataFrame(columns=[
    'id', 'name', 'host_id', 'neighbourhood_id', 'city_id', 'latitude', 'longitude',
    'room_type', 'property_type', 'price', 'minimum_nights', 'maximum_nights', 'license'
])

# Listings description table
proc_listings_description = pd.DataFrame(columns=[
    'listing_id', 'description', 'picture_url', 'accommodates', 'bathrooms', 'bathrooms_text',
    'bedrooms', 'beds', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
    'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm'
])

# Listings amenities table
proc_listings_amenities = pd.DataFrame(columns=['listing_id', 'amenity'])

# Listings availability table
proc_listings_availability = pd.DataFrame(columns=[
    'listing_id', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
    'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
    'has_availability', 'availability_30', 'availability_60', 'availability_90', 'availability_365',
    'instant_bookable'
])

# Listings reviews table
proc_listings_reviews = pd.DataFrame(columns=[
    'listing_id', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
    'first_review', 'last_review', 'review_scores_rating', 'review_scores_accuracy',
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
    'review_scores_location', 'review_scores_value', 'reviews_per_month'
])

# Neighborhood groups table
proc_neighborhood_groups = pd.DataFrame(columns=['id', 'name'])

# Neighborhoods table
proc_neighborhoods = pd.DataFrame(columns=[
    'id', 'name', 'neighborhood_group_id', 'city_id'
])

# Countries table
proc_countries = pd.DataFrame(columns=['iso_3166_1', 'name'])

# Cities table
proc_cities = pd.DataFrame(columns=['id', 'name', 'country_code'])

# Hosts table
proc_hosts = pd.DataFrame(columns=[
    'id', 'url', 'name', 'host_since', 'location', 'host_about', 'response_time', 'response_rate',
    'acceptance_rate', 'is_superhost', 'thumbnail_url', 'picture_url', 'neighborhood',
    'listings_count', 'total_listings_count', 'verifications', 'has_profile_pic',
    'identity_verified', 'calculated_host_listings_count',
    'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms'
])

proc_hosts = listings_gz[["host_url", "host_name", "host_since", "host_location", "host_about", "host_response_time", "host_response_rate", "host_acceptance_rate", 
                            "host_is_superhost", "host_thumbnail_url", "host_picture_url", "host_listings_count", "host_total_listings_count", "host_verifications",
                            "host_has_profile_pic", "host_identity_verified", "calculated_host_listings_count", "calculated_host_listings_count_entire_homes",
                            "calculated_host_listings_count_private_rooms", "calculated_host_listings_count_shared_rooms"]]

proc_hosts.to_csv('proc_host.csv', index=False)


# Users table
proc_users = pd.DataFrame(columns=['id', 'name'])

# Reviews table
proc_reviews = pd.DataFrame(columns=['id', 'listing_id', 'date', 'reviewer_id', 'comments'])

In [174]:
# Create a dictionary to map information from listings to the new dataframes
listings_info = {
    'proc_listings_description': {
        'listings_id': listings_gz['id'],
        'description': listings_gz['description'],
        'picture_url': listings_gz['picture_url'],
        'accommodates': listings_gz['accommodates'],
        'bathrooms': listings_gz['bathrooms'],
        'bathrooms_text': listings_gz['bathrooms_text'],
        'bedrooms': listings_gz['bedrooms'],
        'beds': listings_gz['beds'],
        'minimum_minimum_nights': listings_gz['minimum_minimum_nights'],
        'maximum_minimum_nights': listings_gz['maximum_minimum_nights'],
        'minimum_maximum_nights': listings_gz['minimum_maximum_nights'],
        'maximum_maximum_nights': listings_gz['maximum_maximum_nights'],
        'minimum_nights_avg_ntm': listings_gz['minimum_nights_avg_ntm'],
        'maximum_nights_avg_ntm': listings_gz['maximum_nights_avg_ntm']
    },
    'proc_listings': {
        'id': listings_gz['id'],
        'name': listings_gz['name'],
        'host_id': listings_gz['host_id'],
        'neighbourhood_id': listings_gz['neighbourhood_cleansed'],
        'city_id': listings_gz['city'],
        'latitude': listings_gz['latitude'],
        'longitude': listings_gz['longitude'],
        'room_type': listings_gz['room_type'],
        'property_type': listings_gz['property_type'],
        'price': listings_gz['price'],
        'minimum_nights': listings_gz['minimum_nights'],
        'maximum_nights': listings_gz['maximum_nights'],
        'license': listings_gz['license']
    }, 
    'proc_listings_amenities': {
        'listing_id': listings_gz['id'],
        'amenity': listings_gz['amenities']
    }
}
# Map the dictionary to the new dataframes
for key, value in listings_info.items():
    for col, series in value.items():
        if key == 'proc_listings_description':
            proc_listings[col] = series
        elif key == 'proc_listings':
            proc_listings[col] = series
        elif key == 'proc_listings_amenities':
            proc_listings_amenities[col] = series

# Create a dictionary to map information from calendar to the new dataframes
calendar_info = {
    'proc_calendar': {
        'listing_id': calendar['listing_id'],
        'date': calendar['date'],
        'available': calendar['available'],
        'price': calendar['price'],
        'adjusted_price': calendar['adjusted_price'],
        'minimum_nights': calendar['minimum_nights'],
        'maximum_nights': calendar['maximum_nights']
    }
}
# Map the dictionary to the new dataframes
for key, value in calendar_info.items():
    for col, series in value.items():
        if key == 'proc_calendar':
            proc_calendar[col] = series

neighbourhoods_info = {
    'proc_neighborhoods': {
        'name': neighbourhoods['neighbourhood'],
        'neighborhood_group_id': neighbourhoods['neighbourhood_group'],
        'city_id': neighbourhoods['city'],
    }
}

# Map the dictionary to the new dataframes
for key, value in neighbourhoods_info.items():
    for col, series in value.items():
        if key == 'proc_neighborhoods':
            proc_neighborhoods[col] = series
    

In [175]:
proc_listings_amenities.to_csv('proc_listings_amenities.csv', index=False)
proc_listings_description.to_csv('proc_listings_description.csv', index=False)


In [176]:
cities = ['barcelona', 'madrid', 'malaga', 'mallorca', 'menorca', 'sevilla', 'valencia']
proc_cities['id'] = range(1, len(cities) + 1)
proc_cities['name'] = cities
proc_cities['country_code'] = 'ES'

proc_cities.to_csv('proc_cities.csv', index=False)

In [177]:
proc_countries['iso_3166_1'] = 'ES'
proc_countries['name'] = 'Spain'

proc_countries.to_csv('proc_countries.csv', index=False)

In [178]:
proc_neighborhood_groups['name'] = neighbourhoods['neighbourhood_group'].unique()
null = proc_neighborhood_groups[proc_neighborhood_groups['name'].isnull()].index
proc_neighborhood_groups.drop(null, inplace=True)
proc_neighborhood_groups['id'] = range(1, len(proc_neighborhood_groups) + 1)

proc_neighborhood_groups.to_csv('proc_neighborhood_groups.csv', index=False)

In [179]:
proc_neighborhoods['id'] = range(1, len(neighbourhoods) + 1)
mapping = dict(proc_neighborhood_groups[['name', 'id']].values)
proc_neighborhoods['neighborhood_group_id'] = proc_neighborhoods['neighborhood_group_id'].map(mapping)
proc_neighborhoods['neighborhood_group_id'] = proc_neighborhoods['neighborhood_group_id'].fillna(0)
proc_neighborhoods['neighborhood_group_id'] = proc_neighborhoods['neighborhood_group_id'].astype(int)
proc_neighborhoods['neighborhood_group_id'] = proc_neighborhoods['neighborhood_group_id'].replace(0, np.nan)

proc_neighborhoods.to_csv('proc_neighborhoods.csv', index=False)

In [180]:
proc_listings['neighbourhood_id'] = proc_listings['neighbourhood_id'].map(dict(proc_neighborhoods[['name', 'id']].values))

proc_listings.to_csv('proc_listings.csv', index=False)

In [181]:
boolean_map = {'t': True, 'f': False}
proc_calendar['available'] = proc_calendar['available'].map(boolean_map)
proc_calendar['price'] = proc_calendar['price'].str.replace('$', '').str.replace(',', '').astype(float)
proc_calendar['adjusted_price'] = proc_calendar['adjusted_price'].str.replace('$', '').str.replace(',', '').astype(float)

proc_calendar.to_csv('proc_calendar.csv', index=False)

KeyboardInterrupt: 